## Aufgabe: 

Trainiere ein Modell basierend auf dem VGG16-Modell mithilfe von Transfer Learning, welches Katzen und Hunden unterscheiden kann!



## Herunterladen der Daten

In [1]:
import requests
import numpy as np
import math
import os
import zipfile
from tqdm import tqdm_notebook as tqdm 

if not os.path.exists("data"):
    os.mkdir("data")
    
if not os.path.exists(os.path.join("data", "PetImages")):
    url = "https://downloads.codingcoursestv.eu/037%20-%20neuronale%20netze/PetImages.zip"
    # Streaming, so we can iterate over the response.
    r = requests.get(url, stream=True)

    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0)); 
    block_size = 1024
    
    print("Downloading...")
    with open(os.path.join("data", "PetImages.zip"), 'wb') as f:
        for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size), unit='KB', unit_divisor=1024, unit_scale=True):
            f.write(data)
            
    print("Download completed")
    print("Extracting...")
    
    zip_ref = zipfile.ZipFile(os.path.join("data", "PetImages.zip"), 'r')
    zip_ref.extractall(os.path.join("data"))
    zip_ref.close()
    
    print("Done!")
else:
    print("Die Dateien existieren bereits")


Die Dateien existieren bereits


## Einlesen der Daten

In [2]:
import numpy as np
from PIL import Image

def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
    
    # Limit to 1000 files for faster processing
    #files = files[:1000]
    
    images = []
    # Loop over all files in folder with tqdm to show progress
    for file in tqdm(files):
        try:
            image = Image.open(os.path.join(path, file))

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.resize
            image = image.resize((224, 224), Image.LANCZOS)

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.convert
            image = image.convert("RGB")
            # Convert to numpy array
            image = np.asarray(image)
            # Append to list
            images.append(image)
        except OSError:
            pass
    
    return images
    
cats = read_images(os.path.join("data", "PetImages", "Cat"))
dogs = read_images(os.path.join("data", "PetImages", "Dog"))

/var/folders/by/l34wfpgn2v11pqz8wlmb4v080000gn/T/ipykernel_30326/1078906060.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(files):


  0%|          | 0/12500 [00:00<?, ?it/s]

  0%|          | 0/12500 [00:00<?, ?it/s]

/Users/alex/anaconda3/envs/ml/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


## Preprocessing der Bilddaten von Dogs and Cats

### Convert to numpy arrays

In [3]:
dogs = np.array(dogs)
cats = np.array(cats)

# Combine dogs and cats to one dataset
X = np.concatenate((dogs, cats), axis=0)
print("X shape:", X.shape)


X shape: (24998, 224, 224, 3)


### Create labels

Dogs = 0 and Cats = 1


In [4]:
# Dogs = 0, Cats = 1
y_dogs = np.zeros(len(dogs))
y_cats = np.ones(len(cats))

# Combine labels
y = np.concatenate((y_dogs, y_cats), axis=0)
y = y.reshape(-1, 1)
print("y shape:", y.shape)

del y_dogs, y_cats

y shape: (24998, 1)


## Transfer Learing mit VGG16

In [5]:
from keras.applications.vgg16 import VGG16, preprocess_input

X = preprocess_input(X, mode='tf')

Vortrainiertes VGG16-Modell wird zum Preprocessing der X-Daten genutzt um Modell-Training zu vereinfachen 

In [6]:
vgg16_model = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg16_model.trainable = False
#vgg16_model.summary()

X_after_vgg = vgg16_model.predict(X, verbose=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2520s 3s/step


Input des nachgeschalteten Modells erhalten so eine andere Dimension der Input_shape

In [7]:
X_after_vgg.shape

(24998, 7, 7, 512)

### Shuffle der Daten

Shuffle der Daten damit nicht alle Hunde und Katzen nacheinander kommen


In [8]:
from sklearn.utils import shuffle

# Shuffle der Daten damit nicht alle Hunde und Katzen nacheinander kommen
X_after_vgg, y = shuffle(X_after_vgg, y)

## Model after VGG16

In [9]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam

model = Sequential()

# Model layers with VGG16-Output as Input
model.add(Flatten(input_shape=(7, 7, 512)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

/Users/alex/anaconda3/envs/ml/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    25,691,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │        16,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,707,553 (98.07 MB)

 Trainable params: 25,707,553 (98.07 MB)

 Non-trainable params: 0 (0.00 B)

## Model Training durch Transfer Learning

In [10]:
model.fit(X_after_vgg, y, epochs=10, batch_size=32, validation_split=0.2, shuffle=True)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 62ms/step - accuracy: 0.9641 - loss: 0.1360 - val_accuracy: 0.9804 - val_loss: 0.0568
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 68ms/step - accuracy: 0.9896 - loss: 0.0253 - val_accuracy: 0.9800 - val_loss: 0.0656
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 69ms/step - accuracy: 0.9950 - loss: 0.0103 - val_accuracy: 0.9838 - val_loss: 0.0648
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 70ms/step - accuracy: 0.9978 - loss: 0.0071 - val_accuracy: 0.9824 - val_loss: 0.0748
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 45s 72ms/step - accuracy: 0.9972 - loss: 0.0087 - val_accuracy: 0.9834 - val_loss: 0.0687
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 71ms/step - accuracy: 0.9977 - loss: 0.0070 - val_accuracy: 0.9866 - val_loss: 0.0731
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 70ms/step - accuracy: 0.9983 - loss: 0.0054 - val_accuracy: 0.9868 - val_loss: 0.0708
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 69ms/step - accuracy: 0.9985 - loss: 0.0046 - 

## Abspeichern des trainierten Modells

In [11]:
# Abspeichern des trainierten Modells
model.save(os.path.join("data", "dogs_vs_cats_VGG16model.keras"))